# Use openCV to identify facial features.
1. Identify the eyes, mouth and face from an image. 
2. Detect the iris color, lip color and skin color
3. Use openai to determine color season from image.


Import opencv library.

In [1]:
import cv2

Create face, eye and mouth classifiers that are pre-trained.

In [2]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

Apply the classifiers to the test images to extract the rectangular regions of the features.

In [3]:
test_image = cv2.imread("minette.jpg")
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

In [29]:
test_image = cv2.imread("minette.jpg")
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# faces = face_classifier.detectMultiScale(test_image, 1.8, 4)
# for (x,y,w,h) in faces:
#     cv2.rectangle(test_image, (x,y), (w+x,h+y), (0,0,225), 2)
#     roi_gray = gray_image[y:y+h, x:x+w]
#     roi_color = test_image[y:y+h, x:x+w]

# mouth = mouth_classifier.detectMultiScale(gray_image, 1.8, 28)
# for (x,y,w,h) in mouth:
#     cv2.rectangle(test_image, (x,y), (w+x,h+y), (0,0,225), 2)

eyes = eye_classifier.detectMultiScale(gray_image)
for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(test_image, (ex,ey), (ew+ex,eh+ey), (0,0,225), 2)
    eye_roi_gray = gray_image[ey:ey + eh, ex:ex + ew]
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(eye_roi_gray, (11, 11), 0)
    
    # Detect larger circles using Hough Circle Transform
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                               param1=50, param2=30, minRadius=20, maxRadius=50)
    
    # Draw detected larger circles
    if circles is not None:
        circles = circles[0]  # Convert circles to numpy array
        circles = circles.round().astype("int")  # Round the circles coordinates and convert to integers
        for (cx, cy, cr) in circles:
            # Adjust the coordinates to the entire image space
            cx += ex
            cy += ey
            # Draw larger circles
            cv2.circle(test_image, (cx, cy), cr, (0, 255, 0), 2)
            
            # Extract the region around the larger circle
            x = max(cx - cr, 0)
            y = max(cy - cr, 0)
            w = min(2 * cr, test_image.shape[1] - x)
            h = min(2 * cr, test_image.shape[0] - y)
            circle_region_gray = gray_image[y:y + h, x:x + w]
            
            # Apply Gaussian blur to the extracted region
            blurred_circle_region = cv2.GaussianBlur(circle_region_gray, (11, 11), 0)
            
            # Detect smaller circles (iris) within the extracted region
            iris_circles = cv2.HoughCircles(blurred_circle_region, cv2.HOUGH_GRADIENT, dp=1, minDist=10,
                                             param1=50, param2=30, minRadius=5, maxRadius=20)
            
            # Draw detected smaller circles (iris)
            if iris_circles is not None:
                iris_circles = iris_circles[0]  # Convert circles to numpy array
                iris_circles = iris_circles.round().astype("int")  # Round the circles coordinates and convert to integers
                for (icx, icy, icr) in iris_circles:
                    # Adjust the coordinates to the entire image space
                    icx += x
                    icy += y
                    # Draw smaller circles (iris)
                    cv2.circle(test_image, (icx, icy), icr, (0, 0, 255), 2)
                    
cv2.imwrite("iris_detection.png", test_image)

True